In [6]:
import os
import pickle
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

import plotly.express as px
import plotly.graph_objects as go

from src.data_load import load_tables, load_instance, load_distances
from src.metrics import  collect_results_to_df, compute_metrics_with_moves
from src.alpha_tuning_utils import compute_baseline_metrics, collect_alpha_metrics, \
    add_aggregated_totals, alphas, iterations_nums, metrics
from src.experimentation_config import instance_map, fechas_dict

data_path = '../data'

instance = 'instAS1'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
distance_method = 'haversine'      # Options: ['precalced', 'haversine']

instance_type = instance_map[instance]
fechas = fechas_dict[instance]

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df = load_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, 'osrm', instance)

# Description

This notebook provides an analysis of the experimentation of alpha calibration with the three different objectives for the calibration: 

- hybrid
- extra_time
- driver_move_distance

# Process results data

In [7]:
baseline_df = compute_baseline_metrics(
    data_path=data_path, 
    instance=instance, 
    fechas=fechas, 
    valid_cities=valid_cities, 
    dist_dict=dist_dict
)

metrics_df = pd.DataFrame()
for metric in metrics:
    met_df = collect_alpha_metrics(
                                    data_path=data_path,
                                    alphas=alphas,
                                    instance=instance,
                                    iterations_nums=iterations_nums,
                                    dist_method='haversine',
                                    optimization=metric
                                   )
    met_df['metric'] = metric
    metrics_df = pd.concat([metrics_df, met_df])

metrics_df = metrics_df.join(baseline_df.set_index("city"), on="city")
metrics_w_totals_df = add_aggregated_totals(metrics_df, group_by=["alpha", "num_iterations", 'metric'])


In [8]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

def plot_metrics_vs_alpha(metrics_df):
    def _plot(num_iter, city, perf_metric):
        # Filter by iteration and city
        df = metrics_df[(metrics_df["num_iterations"] == num_iter) & (metrics_df["city"] == city)]
        if df.empty:
            print("⚠️ No data for this selection")
            return

        # --- Main line plot: one line per optimization metric ---
        fig = px.line(
            df,
            x="alpha",
            y=perf_metric,       # performance metric to visualize
            color="metric",      # which optimization was used
            title=f"{perf_metric} vs. Alpha<br>iterations={num_iter}, city={city}",
            markers=True
        )

        # --- Add baseline line ---
        baseline_col = f"{perf_metric}_baseline"
        if baseline_col in df.columns:
            baseline_value = df[baseline_col].iloc[0]  # constant for all alphas
            fig.add_trace(
                go.Scatter(
                    x=df["alpha"].unique(),
                    y=[baseline_value] * df["alpha"].nunique(),
                    mode="lines",
                    line=dict(color="orange", dash="dash"),
                    name="Baseline"
                )
            )

        # --- Layout ---
        fig.update_layout(
            xaxis_title="Alpha",
            yaxis_title=perf_metric,
            template="plotly_white",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.3,
                xanchor="center",
                x=0.5
            )
        )

        fig.show()

    interact(
        _plot,
        num_iter=widgets.Dropdown(
            options=sorted(metrics_df["num_iterations"].unique()),
            description="Iterations:"
        ),
        city=widgets.Dropdown(
            options=sorted(metrics_df["city"].unique()),
            description="City:"
        ),
        perf_metric=widgets.Dropdown(
            options=["vt_labors", "driver_extra_time", "driver_move_distance"],
            description="Metric:"
        )
    )


In [ ]:
plot_metrics_vs_alpha(metrics_w_totals_df)

interactive(children=(Dropdown(description='Iterations:', options=(np.int64(10), np.int64(25), np.int64(50), n…